<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/embedding_text_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストファイルをEmbeddingするプログラム

In [ ]:
# テキストをハッシュ(SHA-3)にする関数
import hashlib
def hash_text_sha3(text):
  h = hashlib.sha3_256()
  h.update(text.encode('utf-8'))
  return h.hexdigest()

In [ ]:
# ハッシュの実行テスト
hashed = hash_text_sha3("hello,world!")
print(f"hashed : {hashed}")

## 形態素解析する関数

In [ ]:
# パッケージのインストール
!pip install sudachipy sudachidict_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 7.1 MB/s eta 0:00:00


In [ ]:
# テキストを形態素解析する関数
import sudachipy
from sudachipy import tokenizer
from sudachipy import dictionary

# Sudachiの辞書を読み込む
tokenizer_obj = dictionary.Dictionary().create()

def tokenize(text):
    # 形態素解析を実行
    mode = tokenizer.Tokenizer.SplitMode.C
    tokens = tokenizer_obj.tokenize(text, mode)

    result = []
    for token in tokens:
        # 品詞情報を取得
        part_of_speech = token.part_of_speech()

        # 名詞、動詞、形容詞、または固有名詞の場合にトークンを追加
        if (part_of_speech[0] in ['名詞', '動詞', '形容詞'] or
            (part_of_speech[0] == '名詞' and part_of_speech[1] == '固有名詞')):
            result.append(token.surface())

    return result

In [ ]:
# 形態素解析の関数の実行テスト
text = "LLMにおいて増分学習と転移学習はどのように違っていてどう有用なのか、私は知りたいです。"
token = tokenize(text)
print(token)

['LLM', 'おい', '増分', '学習', '転移', '学習', '違っ', 'い', '有用', '知り']


In [ ]:
# トークナイザー
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.8 MB/s eta 0:00:00


In [ ]:
import tiktoken

def get_tokens(text):
  # エンコーディングの取得
  enc = tiktoken.get_encoding("cl100k_base")
  # エンコードの実行
  tokens = enc.encode(text)
  return tokens

In [ ]:
tokens = get_tokens("こんにちは、世界！")
print(len(tokens))

6


# ファイルを読み込んですべてのファイルパスを返す関数

In [4]:
from google.colab import drive
import os

# Google Driveをマウントする
drive.mount('/content/drive')

# 起点となるフォルダのパスを指定する
base_folder = '/content/drive/MyDrive/PL自動割り当て用プロトデータ'

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # テキストファイルのみ処理する
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

# 起点となるフォルダ内のテキストファイルのパスを取得する
file_paths = get_file_paths(base_folder)

# ファイルパスを表示する
print(len(file_paths))
#for path in file_paths:
#    print(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
548


# ファイルパスからトークンサイズを確認する

In [ ]:
from typing import List

def read_text_files(file_paths: List[str]) -> List[str]:
    texts = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            texts.append(text)
    return texts

In [ ]:
new_array = [file_paths[0]]
print(new_array)
texts = read_text_files(new_array)

['/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-28-23_1.txt']


# テキストのトークンの数を確認する

In [ ]:
# ファイルパスからテキストを取得する
def get_file_content(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            return content
    except IOError:
        print(f"ファイル {file_path} の読み込み中にエラーが発生しました。")
        return None

In [ ]:
content = get_file_content(file_paths[0])
tokens = get_tokens(content)
print(len(tokens))

732


In [ ]:
# ファイルパスのリストをトークン数制限に基づいてグループ化する関数
def split_files_by_token_count(file_paths, max_tokens=8000):
    file_path_groups = []
    current_group = []
    current_token_count = 0

    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            token_count = len(get_tokens(text))

            if current_token_count + token_count <= max_tokens:
                current_group.append(file_path)
                current_token_count += token_count
            else:
                file_path_groups.append(current_group)
                current_group = [file_path]
                current_token_count = token_count

    if current_group:
        file_path_groups.append(current_group)

    return file_path_groups

In [ ]:
file_path_groups = split_files_by_token_count(file_paths)

print(file_path_groups);

[['/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-28-23_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-21_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-35_2.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-47_3.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-16_4.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-28_5.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-44_6.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-01_7.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-18_8.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-38_9.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/r

In [ ]:
print(len(file_path_groups))

45


# CSVに作業者と作業済みファイルのデータを登録する

In [16]:
import csv
import random

# CSVを作成する

def assign_file_info(file_paths, csv_file):
    # CSVファイルが存在しない場合は新規作成し、ヘッダーを書き込む
    if not os.path.exists(csv_file):
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['index', 'file_path', 'worker_id'])

    # 既存のCSVファイルからデータを読み込む
    existing_data = []
    if os.path.exists(csv_file):
        with open(csv_file, mode='r') as file:
            reader = csv.reader(file)
            next(reader)  # ヘッダーをスキップ
            existing_data = list(reader)

    # worker_idごとの優先ファイルパスの設定
    priority_paths = {
        1: 'IT',
        2: 'houmu',
        3: 'qa',
        4: 'quality',
        5: 'r_and_d'
    }

    # 新しいデータを生成
    new_data = []
    for file_path in file_paths:
        index = len(existing_data) + len(new_data)

        # 優先ファイルパスに基づいてworker_idを割り当て
        assigned_worker_id = None
        for worker_id, priority_path in priority_paths.items():
            if priority_path in file_path:
                if random.random() < 0.8:  # 80%の確率で優先割り当て
                    assigned_worker_id = worker_id
                    break

        # 優先割り当てされなかった場合はランダムにworker_idを割り当て
        if assigned_worker_id is None:
            assigned_worker_id = random.randint(1, 5)

        new_data.append([index, file_path, assigned_worker_id])

           # 新しいデータをCSVファイルに追記
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(new_data)

    print(f"Assigned {len(new_data)} files to workers.")

In [17]:
# ファイルパスを取得
csv_file = "./worked_files.csv"

assign_file_info(file_paths, csv_file)

Assigned 548 files to workers.


In [12]:
# フォルダごとに割り当てられた担当者の割合を出す関数
from collections import defaultdict

def analyze_worker_distribution(csv_file):
    # フォルダごとの担当者の割合を格納する辞書
    folder_worker_distribution = {
        'IT': defaultdict(int),
        'houmu': defaultdict(int),
        'qa': defaultdict(int),
        'quality': defaultdict(int),
        'r_and_d': defaultdict(int)
    }

    # CSVファイルからデータを読み込む
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # ヘッダーをスキップ

        # 各行を処理
        for row in reader:
            file_path = row[1]
            worker_id = int(row[2])

            # フォルダごとに担当者の割合を集計
            for folder in folder_worker_distribution.keys():
                if folder in file_path:
                    folder_worker_distribution[folder][worker_id] += 1
                    break

    # 割合を計算して表示
    for folder, worker_counts in folder_worker_distribution.items():
        total_files = sum(worker_counts.values())
        if total_files > 0:
            print(f"Folder: {folder}")
            for worker_id, count in worker_counts.items():
                percentage = count / total_files * 100
                print(f"  Worker {worker_id}: {percentage:.2f}%")
        else:
            print(f"Folder: {folder}")
            print("  No files assigned.")
        print()

In [18]:
analyze_worker_distribution(csv_file)

Folder: IT
  Worker 4: 8.57%
  Worker 1: 84.29%
  Worker 2: 2.86%
  Worker 5: 2.86%
  Worker 3: 1.43%

Folder: houmu
  Worker 1: 5.63%
  Worker 2: 83.10%
  Worker 3: 7.04%
  Worker 4: 2.82%
  Worker 5: 1.41%

Folder: qa
  Worker 3: 82.78%
  Worker 5: 5.26%
  Worker 2: 4.31%
  Worker 1: 4.31%
  Worker 4: 3.35%

Folder: quality
  Worker 4: 76.85%
  Worker 3: 7.41%
  Worker 1: 9.26%
  Worker 2: 2.78%
  Worker 5: 3.70%

Folder: r_and_d
  Worker 5: 80.00%
  Worker 3: 6.67%
  Worker 1: 7.78%
  Worker 4: 3.33%
  Worker 2: 2.22%



# Embedding

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# クライアントの準備
client = OpenAI()

In [ ]:
import numpy as np

def get_embeddings(texts: List[str]) -> List[List[float]]:

  response = client.embeddings.create(
    input=texts,
    model="text-embedding-ada-002"
 )
  return response

In [ ]:
# ファイルパスからファイルを取得する
def get_file_contents(file_paths):
    file_contents = []
    for path in file_paths:
        try:
            with open(path, 'r') as file:
                content = file.read()
                file_contents.append(content)
        except IOError:
            print(f"ファイル {path} の読み込み中にエラーが発生しました。")
    return file_contents

In [ ]:
# ループでEmbeddingの値を格納する
def get_all_embeddings(filePaths):
  embeddings = []
  for filePath in filePaths:
    print(filePath)
    file_contents = get_file_contents(filePath)
    response = get_embeddings(file_contents)
    batch_embeddings = [record.embedding for record in response.data]
    embeddings.extend(batch_embeddings)
  return embeddings

In [ ]:
# npyファイルとして保存する関数
def save_embeddings(embeddings, file_path):
    np.save(file_path, embeddings)

In [ ]:
#contents_1 = get_file_contents(file_path_groups[0])
contents_2 = get_file_contents(file_path_groups[1])
print(len(contents_2))

print(file_path_groups)

10
[['/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-28-23_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-21_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-35_2.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-47_3.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-16_4.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-28_5.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-44_6.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-01_7.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-18_8.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-38_9.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_ag

In [ ]:
# numpy配列に変換
target_embeds = get_all_embeddings(file_path_groups)
target_embeds = np.array(target_embeds).astype("float32")

# Embeddingを保存
save_embeddings(target_embeds,"embeddings_all.npy")


['/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-28-23_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-21_1.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-35_2.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-29-47_3.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-16_4.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-28_5.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-30-44_6.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-01_7.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-18_8.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/response_2024-03-28_07-31-38_9.txt', '/content/drive/MyDrive/PL自動割り当て用プロトデータ/sample_houmu_agm/re

# ベクトルデータベースに追加